In [2]:
# 랭체인 에이전트 툴 구성 : langchain agent + tool + LLM
# LLM에게 상황을 분석하게 하고 -> 필요한 툴을 LLM이 스스로 선택한 후 툴을 실행하고 -> 결과를 반영해
# 다시 답을 만드는 '멀티 턴 자동의사 결정' 로직 구성. ReAct
# ReAct 프롬프트는 "AI가 스스로 판단하고, 도구를 선택하고, 연속된 reasoning을 할 수 있게 만드는 설계도"다.

!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install -U langchain langchain-core langchain-community \
                langchain-google-genai google-genai \
                langchain-chroma sentence-transformers \
                python-dotenv langchain-classic

Found existing installation: langchain 1.1.0
Uninstalling langchain-1.1.0:
  Successfully uninstalled langchain-1.1.0
Found existing installation: langchain-core 1.1.0
Uninstalling langchain-core-1.1.0:
  Successfully uninstalled langchain-core-1.1.0
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langchain-google-genai 3.2.0
Uninstalling langchain-google-genai-3.2.0:
  Successfully uninstalled langchain-google-genai-3.2.0
  Using cached langchain-1.1.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_core-1.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-1.1.0-py3-none-any.whl (101 kB)
Using cached langchain_core-1.1.0-py3-none-any.whl (473 kB)
Using cached langchain_community-0.4.

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
import os
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from dotenv import load_dotenv
from google.colab import userdata
from langchain_core.runnables import RunnablePassthrough

# --- LLM 및 응답 정리 함수 ---

# .env 파일에서 환경 변수(API 키 등) 로드
load_dotenv()
# Gemini API 키를 환경 변수로 설정
os.environ["GOOGLE_GENAI_PROJECT"] = userdata.get('GOOGLE_GENAI_PROJECT')

# LLM 모델
llm = ChatGoogleGenerativeAI(
    temperature=0.2,
    model="gemini-2.5-flash",
    google_api_key=userdata.get('GOOGLE_API_KEY')
)

In [9]:
# 0) 공통 유틸 --LLM이 생성한 응답 문자열을 정리해 주는 헬퍼 함수
# Gemini 2.x/2.5/Flash 계열의 특징 때문. 같은 답변을 2번 반복하는 버그/패턴이 자주 발생함
# LLM 응답을 깔끔하게 정리하는 유틸리티 함수
def clean_answer(t: str) -> str:
    # 1) 입력 객체를 무조건 문자열로 만들고 앞뒤 공백 제거
    t = str(t).strip()  # (ChatMessage 객체일 수도 있어서 str()으로 통일)

    # 2) 전체 문자열이 정확히 두 번 반복된 패턴인지 검사
    #    예: "ABC\nABC\n" 같은 경우 → 절반으로 잘라서 중복 제거
    if len(t) % 2 == 0 and t[:len(t)//2] == t[len(t)//2:]:
        t = t[:len(t)//2].strip() # 앞 절반만 사용

    # 3) 문단 단위로 중복 제거 : "\n\n" 기준으로 문단 나눠서 이미 본 문단은 제외
    out, seen = [], set()
    for p in t.split("\n\n"):
        p = p.strip()
        if p and p not in seen:   # 비어있지 않고 중복되지 않은 문단만 추가
            seen.add(p)
            out.append(p)

    # 4) 최종 문단들을 공백 한 칸으로 연결
    merged = " ".join(out)
    cleaned = merged.replace("**", "").replace("*", "")   # 마크다운 기호 제거 (**, *)
    return cleaned.strip()

# 프롬프트를 받아 LLM을 호출하고, 결과를 clean_answer로 정리하는 공통 함수
def run_llm(prompt: str) -> str:    # 실행 5단계 : Gemini호출, 질문에 대한 답 ------------------------ 5
    resp = llm.invoke(prompt)
    # ChatMessage면 content, 아니면 그대로 문자열을 반환
    return clean_answer(getattr(resp, "content", resp))


# 1) Tools ------------
@tool       # 이 함수를 LangChain Tool로 등록
def math_helper(question: str) -> str:
    """수학 문제를 단계별로 풀이하고 마지막 줄에 정답을 한 번 더 적어 주는 도우미."""
    prompt = (
        "너는 수학 풀이를 잘하는 모범생이야.\n"
        "아래 수학 문제를 단계별로 풀고, 마지막 줄에 정답을 적어 줘.\n\n"
        f"문제: {question}\n"    # 사용자의 질문(수학 문제)을 그대로 끼워 넣기
        "풀이:"
    )
    return run_llm(prompt)     # 공통 run_llm 함수로 LLM 호출 + 정리된 답변 반환
    # 실행 6단계 : Tool 실행 결과가 Agent로 돌아옴 ------------------------------------------------------------------- 6


@tool
def code_helper(question: str) -> str:
    """코딩/프로그래밍 질문에 대해 1)설명 2)예제 코드 3)중요한 포인트 순서로 답하는 도우미."""
    prompt = (
        "너는 전문 프로그래머야.\n"
        "아래 요청에 대해 1)간단한 설명 2)예제 코드 3)중요한 포인트 순서로 답변해 줘.\n\n"
        f"요청: {question}\n"
        "답변:"
    )
    return run_llm(prompt)

@tool
def general_helper(question: str) -> str:     # 실행 4단계 ------------------------------ 4
    """일반 개념/이론을 이해하기 쉽게 3~4문장으로 설명하는 도우미."""
    prompt = (
        "너는 친절한 AI 도우미야.\n"
        "아래 질문에 대해 3~4문장으로 설명해 줘.\n"
        f"질문: {question}\n"
        "답변:"
    )
    return run_llm(prompt)

tools = [math_helper, code_helper, general_helper]  # Agent에게 넘겨줄 Tool 목록


# 2) Agent
prompt = ChatPromptTemplate.from_messages(   # Agent용 전체 프롬프트 템플릿 정의
  [
    (
      "system",     # 시스템 메시지: Agent의 역할과 Tool 사용 규칙 정의
      "너는 사용자의 질문을 보고 적절한 툴을 선택하는 에이전트다.\n"
      "- 수식, 계산, 더하기/빼기/곱하기/나누기, '+', '-', '*', '/' 등이 보이면 math_helper를 사용해.\n"
      "- '함수', '클래스', Python/Java/C언어/JavaScript 등의 단어가 보이면 code_helper를 사용해.\n"
      "- 그 외의 일반적인 개념/이론/설명은 general_helper를 사용해.\n"
      "하지만 툴에서 반환된 텍스트를 그대로 복사해서 출력하지 말고, "
      "툴의 내용을 참고해 최종 답변을 한국어로 깔끔하게 작성해.\n"
    ),
    # 이전 대화 내용을 주입하기 위한 placeholder(자리표시자)
    # (이전까지의 대화 기록(사람 → AI → 사람 → AI...))
    # 이전 대화 히스토리를 넣기 위한 자리. 대화의 연속성과 문맥 유지
    MessagesPlaceholder(variable_name="chat_history"),

    ("human", "{input}"), # 현재 사용자가 입력한 질문이 들어가는 자리
    # ask("3 더하기 5는?") 라고 하면 ("human", "{input}")에 ("human", "3 더하기 5는?")가 됨

    # Agent가 Tool호출등 중간 사고과정을 쌓는 내부 메모. Agent가 스스로 결정 내리기 위한 메모장역할

    MessagesPlaceholder(variable_name="agent_scratchpad"),
  ]
)

# Tool-calling Agent 생성 (LLM + Tools + Prompt 연결)
llm_with_tools = llm.bind_tools(tools)

agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

agent_executor = AgentExecutor( # Agent를 실제로 실행하는 래퍼    # 실행 3단계 : Gemini 호출, 어떤 툴을 사용할지 ------------------ 3
    agent=agent,  # 방금 만든 Agent
    tools=tools,   # 동일한 Tool 목록 전달
    verbose=False,  # True면 내부 Tool 호출 로그를 콘솔에 출력, False면 숨김
    handle_parsing_errors=True
)

In [13]:
# 3) 간단 테스트 래퍼 --------------------
chat_history = []      # 대화 히스토리(간단 버전) 저장용 리스트

def askFunc(q: str):   # 한 번의 질의를 실행하는 헬퍼 함수
    print("\n==============================")
    print("질문:", q)                   # 콘솔에 질문 출력
    result = agent_executor.invoke(    # AgentExecutor를 통해 Agent 호출  # 실행 2단계 ---------------- 2
        {
            "input": q,       # 프롬프트 템플릿의 {input} 에 매핑될 값
            "chat_history": chat_history,  # 이전 턴들의 대화 히스토리
        }
    )
    print("\n[최종 답변]")
    print(result["output"])               # Agent가 최종적으로 생성한 답변 출력  # 실행 7단계 --------------- 7
    chat_history.append(("human", q))     # 히스토리에 사용자 질문 추가   # 실행 8단계 --------------- 8
    chat_history.append(("ai", result["output"]))    # 히스토리에 AI 답변 추가



q1 = "3 더하기 5 곱하기 2는 얼마인가?"
askFunc(q1)     # 실행 1단계 ------------ 1

q2 = "자바로 숫자들의 평균을 구하는 코드를 보여줘."
askFunc(q2)

q3 = "가을과 겨울의 차이는 무엇인가?"
askFunc(q3)


질문: 3 더하기 5 곱하기 2는 얼마인가?

[최종 답변]
주어진 수학 문제 "3 더하기 5 곱하기 2"를 풀어보겠습니다.

수학에서는 곱셈이 덧셈보다 우선순위가 높으므로, 먼저 5 곱하기 2를 계산합니다.
5 × 2 = 10

그 다음, 이 결과에 3을 더합니다.
3 + 10 = 13

따라서 정답은 13입니다.

질문: 자바로 숫자들의 평균을 구하는 코드를 보여줘.

[최종 답변]
자바에서 숫자들의 평균을 구하는 방법에 대해 설명해 드릴게요.

### 1) 간단한 설명
숫자들의 평균은 모든 숫자의 총합을 구한 다음, 그 총합을 숫자의 개수로 나누는 기본적인 수학 연산입니다. 자바에서는 배열이나 컬렉션에 담긴 숫자들을 반복문으로 순회하며 합계를 계산하고, 그 합계를 배열/컬렉션의 크기로 나누어 평균을 얻을 수 있습니다. 정확한 계산을 위해 `double` 타입을 사용하는 것이 중요합니다.

### 2) 예제 코드
다음은 `double` 타입 배열에 담긴 숫자들의 평균을 계산하는 자바 코드 예제입니다. 빈 배열이나 `null` 입력에 대한 예외 처리도 포함되어 있습니다.

```java
import java.util.Arrays;
import java.util.List;
import java.util.OptionalDouble;

public class AverageCalculator {
    /**
     * double 배열의 평균을 계산합니다.
     * 빈 배열이나 null이 입력되면 IllegalArgumentException을 발생시킵니다.
     *
     * @param numbers 평균을 계산할 숫자 배열
     * @return 숫자들의 평균 값
     * @throws IllegalArgumentException numbers가 null이거나 비어있을 경우
     */
    public static double calculateAverage(double[] numbers) {
        // 1. 입력 유효성 검사 